In [10]:
import json
import mysql.connector
import pandas as pd

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo",
    charset="latin1"
)

df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df

,id,username,description,date,edits,reviews,comments,taxonomy
0,1,arekkas,"[[{""col"":24,""content"":""As MSc Computer Science...",2014-03-01 20:36:21,163.0,172.0,78.0,784.0
1,2,devuser,NULL,2014-03-01 20:36:21,2.0,1.0,4.0,3.0
2,4,Deleted,NULL,2014-03-01 20:36:21,161.0,187.0,7.0,17.0
3,6,Legacy,NULL,2014-03-01 20:36:21,6986.0,6988.0,0.0,1908.0
4,9,David,NULL,2014-03-01 20:36:32,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
23458,233569,Brindha,NULL,2021-12-21 13:07:25,0.0,0.0,0.0,0.0
23459,233590,Leo12345678,NULL,2021-12-21 17:27:02,0.0,0.0,0.0,0.0
23460,233607,alanwats,NULL,2021-12-22 08:17:01,0.0,0.0,0.0,0.0
23461,233608,danddclothing,NULL,2021-12-22 08:19:36,0.0,0.0,0.0,0.0


In [11]:
import json

def has_link(description):
    if description == None or description == "NULL":
        return False
    
    return "http://" in description or "https://" in description

df["has_link"] = df["description"].map(has_link)
df

,id,username,description,date,edits,reviews,comments,taxonomy,has_link
0,1,arekkas,"[[{""col"":24,""content"":""As MSc Computer Science...",2014-03-01 20:36:21,163.0,172.0,78.0,784.0,True
1,2,devuser,NULL,2014-03-01 20:36:21,2.0,1.0,4.0,3.0,False
2,4,Deleted,NULL,2014-03-01 20:36:21,161.0,187.0,7.0,17.0,False
3,6,Legacy,NULL,2014-03-01 20:36:21,6986.0,6988.0,0.0,1908.0,False
4,9,David,NULL,2014-03-01 20:36:32,0.0,0.0,1.0,0.0,False
...,...,...,...,...,...,...,...,...,...
23458,233569,Brindha,NULL,2021-12-21 13:07:25,0.0,0.0,0.0,0.0,False
23459,233590,Leo12345678,NULL,2021-12-21 17:27:02,0.0,0.0,0.0,0.0,False
23460,233607,alanwats,NULL,2021-12-22 08:17:01,0.0,0.0,0.0,0.0,False
23461,233608,danddclothing,NULL,2021-12-22 08:19:36,0.0,0.0,0.0,0.0,False


In [12]:
df = df[df["has_link"] == True]
df.sort_values(["edits", "reviews", "comments", "taxonomy", "id"],
               ascending=False, inplace=True)
df

<ipython-input-12-2308e75e7a58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(["edits", "reviews", "comments", "taxonomy", "id"],


,id,username,description,date,edits,reviews,comments,taxonomy,has_link
4279,73435,kathongi,"{""plugin"":""rows"",""state"":[{""plugin"":""multimedi...",2017-04-03 08:19:03,6873.0,8423.0,462.0,3268.0,True
5236,87602,Karin,"{""plugin"":""rows"",""state"":[{""plugin"":""layout"",""...",2017-10-17 08:54:11,4041.0,7497.0,89.0,1039.0,True
13593,163773,LinaMaria,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2020-04-23 16:20:50,3567.0,3524.0,10.0,2479.0,True
1329,27693,Nish,"{""plugin"":""rows"",""state"":[{""plugin"":""text"",""st...",2014-08-25 09:44:46,2757.0,4388.0,971.0,2262.0,True
4197,70932,metzgaria,"{""plugin"":""rows"",""state"":[{""plugin"":""image"",""s...",2017-03-13 10:53:59,2377.0,3218.0,221.0,1541.0,True
...,...,...,...,...,...,...,...,...,...
3052,63554,emawadson3,"[[{""col"":24,""content"":""Searching for the right...",2016-09-19 09:54:38,0.0,0.0,0.0,0.0,True
3010,62295,simranrr6,"[[{""col"":24,""content"":""[packers and movers in ...",2016-09-05 06:59:51,0.0,0.0,0.0,0.0,True
2957,60296,tasya,"[[{""col"":9,""content"":""""},{""col"":15,""content"":""...",2016-07-29 05:15:08,0.0,0.0,0.0,0.0,True
2789,57690,dekyuyun,"[[{""col"":12,""content"":""**[tai game](http://tha...",2016-05-12 08:13:40,0.0,0.0,0.0,0.0,True


In [13]:
df[["id", "username", "edits", "comments", "reviews", "taxonomy"]].to_csv("/tmp/potential_bots.csv")

In [29]:
d = df[(df["edits"] + df["comments"] + df["reviews"] + df["taxonomy"] == 0) & df["has_link"]].copy()
d.sort_values(["date"], inplace=True)
ids = list(d["id"])

current_index = 120

from itertools import count

for i in count(6):
    i = min(i, 9)
    next_index = min(len(ids), current_index + 2**i)
    
    display(Markdown("### %s - %s" % (current_index, next_index)))
    
    k = ", ".join(map(str, ids[current_index:next_index]))
    
    print("""mutation {
  user {
    deleteBots(input: {botIds: [%s]}) {
      success
    }
  }
}""" % k)
    
    if next_index >= len(ids):
        break
    else:
        current_index = next_index

### 120 - 184

mutation {
  user {
    deleteBots(input: {botIds: [138220, 138538, 138736, 139044, 139047, 139051, 139171, 139660, 139701, 139755, 139756, 139812, 139986, 140113, 140199, 140200, 140201, 140294, 140295, 140297, 140312, 140327, 140440, 140453, 140504, 140505, 140689, 140690, 140924, 140925, 141029, 141221, 141225, 141285, 141286, 141332, 141466, 141473, 141559, 141571, 141815, 142042, 142043, 142058, 142176, 142223, 142224, 142290, 142292, 142377, 143074, 143084, 143347, 143985, 143993, 144106, 144518, 144519, 144685, 144692, 144728, 144733, 144909, 144922]}) {
      success
    }
  }
}


### 184 - 312

mutation {
  user {
    deleteBots(input: {botIds: [145048, 145056, 145336, 145351, 145377, 145380, 145394, 145401, 145411, 145453, 145575, 145627, 145628, 145682, 145715, 145789, 145790, 145792, 145808, 145809, 145861, 145880, 145996, 146016, 146020, 146023, 146026, 146035, 146043, 146099, 146108, 146144, 146147, 146159, 146163, 146167, 146194, 146237, 146239, 146259, 146261, 146387, 146686, 146801, 146947, 147061, 147062, 147087, 147089, 147096, 147102, 147118, 147145, 147148, 147175, 147210, 147239, 147250, 147281, 147316, 147317, 147321, 147449, 147468, 147475, 147486, 147490, 147504, 147508, 147513, 147551, 147626, 147627, 147646, 147655, 147656, 147738, 147746, 147747, 147748, 147763, 147766, 147779, 147783, 147810, 147811, 147817, 147822, 147837, 147946, 147947, 147961, 147964, 147965, 147966, 147987, 147989, 147990, 148065, 148071, 148076, 148097, 148122, 148215, 148229, 148267, 148268, 148271, 148348, 148364, 148385, 148390, 148434, 148435, 148437, 148460, 148488, 148489, 1484

### 312 - 568

...

mutation {
  user {
    deleteBots(input: {botIds: [215957, 216139, 216141, 216146, 216158, 216166, 216183, 216190, 216193, 216286, 216298, 216299, 216300, 216301, 216302, 216304, 216305, 216308, 216312, 216318, 216320, 216322, 216327, 216343, 216370, 216372, 216482, 216484, 216486, 216487, 216488, 216491, 216496, 216497, 216502, 216505, 216506, 216507, 216508, 216510, 216511, 216512, 216514, 216515, 216516, 216519, 216528, 216539, 216540, 216541, 216564, 216566, 216567, 216569, 216570, 216571, 216572, 216573, 216574, 216591, 216595, 216596, 216690, 216691, 216692, 216693, 216695, 216713, 216798, 216899, 216916, 216939, 216940, 216958, 216959, 216961, 216962, 216974, 217029, 217055, 217152, 217160, 217161, 217166, 217170, 217172, 217174, 217227, 217336, 217374, 217516, 217518, 217525, 217541, 217552, 217554, 217588, 217612, 217623, 217645, 217658, 217698, 217716, 217718, 217812, 217813, 217862, 217884, 217897, 217944, 217946, 217960, 218002, 218004, 218008, 218018, 218022, 218026, 2180

### 5688 - 6200

mutation {
  user {
    deleteBots(input: {botIds: [221692, 221695, 221696, 221697, 221698, 221699, 221707, 221736, 221753, 221790, 221814, 221921, 221925, 221932, 221947, 221948, 221949, 221950, 221957, 221960, 221961, 221964, 221965, 221967, 221968, 221969, 221971, 221972, 221990, 222021, 222022, 222023, 222025, 222026, 222035, 222036, 222069, 222079, 222080, 222081, 222082, 222083, 222085, 222087, 222088, 222090, 222092, 222093, 222097, 222125, 222126, 222145, 222146, 222148, 222156, 222159, 222161, 222220, 222222, 222235, 222237, 222243, 222246, 222247, 222270, 222293, 222300, 222314, 222315, 222319, 222321, 222325, 222327, 222333, 222334, 222388, 222413, 222420, 222595, 222599, 222601, 222606, 222620, 222659, 222673, 222675, 222676, 222683, 222684, 222696, 222697, 222698, 222699, 222707, 222711, 222712, 222713, 222717, 222719, 222752, 222753, 222754, 222755, 222761, 222762, 222763, 222765, 222772, 222773, 222775, 222776, 222780, 222800, 222801, 222811, 222818, 222831, 222832, 2228

### 6200 - 6353

mutation {
  user {
    deleteBots(input: {botIds: [226368, 226369, 226371, 226374, 226405, 226409, 226427, 226430, 226453, 226455, 226458, 226459, 226465, 226466, 226467, 226468, 226469, 226508, 226509, 226510, 226512, 226515, 226555, 226578, 226701, 226703, 226730, 226731, 226737, 226756, 226775, 226814, 226818, 226819, 226824, 226861, 226862, 226865, 226876, 226889, 226890, 226894, 226895, 226896, 226908, 226910, 226917, 226919, 226937, 227008, 227013, 227040, 227041, 227042, 227044, 227084, 227085, 227105, 227144, 227176, 227181, 227183, 227202, 227227, 227253, 227269, 227313, 227322, 227370, 227379, 227448, 227465, 227466, 227467, 227468, 227476, 227485, 227486, 227488, 227534, 227535, 227537, 227541, 227542, 227543, 227545, 227548, 227557, 227571, 227582, 227585, 227586, 227611, 227614, 227615, 227617, 227623, 227635, 227636, 227651, 227655, 227656, 227669, 227685, 227693, 227704, 227722, 227723, 227735, 227739, 227742, 227743, 227746, 227750, 227756, 227757, 227762, 227766, 2277